In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spam').getOrCreate()

In [2]:
data = spark.read.csv('/FileStore/tables/SMSSpamCollection', inferSchema=True, sep='\t')

In [3]:
data.show()

+----+--------------------+
 _c0| _c1|
+----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
spam|WINNER!! As a val...|
spam|Had your mobile 1...|
 ham|I'm gonna be home...|
spam|SIX chances to wi...|
spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows

In [4]:
data = data.withColumnRenamed('_c0', 'class').withColumnRenamed('_c1', 'text')

In [5]:
data.show()

+-----+--------------------+
class| text|
+-----+--------------------+
 ham|Go until jurong p...|
 ham|Ok lar... Joking ...|
 spam|Free entry in 2 a...|
 ham|U dun say so earl...|
 ham|Nah I don't think...|
 spam|FreeMsg Hey there...|
 ham|Even my brother i...|
 ham|As per your reque...|
 spam|WINNER!! As a val...|
 spam|Had your mobile 1...|
 ham|I'm gonna be home...|
 spam|SIX chances to wi...|
 spam|URGENT! You have ...|
 ham|I've been searchi...|
 ham|I HAVE A DATE ON ...|
 spam|XXXMobileMovieClu...|
 ham|Oh k...i'm watchi...|
 ham|Eh u remember how...|
 ham|Fine if thats th...|
 spam|England v Macedon...|
+-----+--------------------+
only showing top 20 rows

In [6]:
from pyspark.sql.functions import length

In [7]:
data = data.withColumn('length', length(data['text']))

In [8]:
data.show()

+-----+--------------------+------+
class| text|length|
+-----+--------------------+------+
 ham|Go until jurong p...| 111|
 ham|Ok lar... Joking ...| 29|
 spam|Free entry in 2 a...| 155|
 ham|U dun say so earl...| 49|
 ham|Nah I don't think...| 61|
 spam|FreeMsg Hey there...| 147|
 ham|Even my brother i...| 77|
 ham|As per your reque...| 160|
 spam|WINNER!! As a val...| 157|
 spam|Had your mobile 1...| 154|
 ham|I'm gonna be home...| 109|
 spam|SIX chances to wi...| 136|
 spam|URGENT! You have ...| 155|
 ham|I've been searchi...| 196|
 ham|I HAVE A DATE ON ...| 35|
 spam|XXXMobileMovieClu...| 149|
 ham|Oh k...i'm watchi...| 26|
 ham|Eh u remember how...| 81|
 ham|Fine if thats th...| 56|
 spam|England v Macedon...| 155|
+-----+--------------------+------+
only showing top 20 rows

In [9]:
data.groupBy('class').mean().show()

+-----+-----------------+
class| avg(length)|
+-----+-----------------+
 ham| 71.4545266210897|
 spam|138.6706827309237|
+-----+-----------------+

In [10]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF, StringIndexer, VectorAssembler

In [11]:
tokenizer = Tokenizer(inputCol='text', outputCol='token_text')
stop_remove = StopWordsRemover(inputCol='token_text', outputCol='stop_token')
count_vec = CountVectorizer(inputCol='stop_token', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_numeric = StringIndexer(inputCol='class', outputCol='label')

clean_up = VectorAssembler(inputCols=['tf_idf', 'length'], outputCol='features')

In [12]:
from pyspark.ml.classification import NaiveBayes

In [13]:
nb = NaiveBayes()

In [14]:
from pyspark.ml import Pipeline

In [15]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_numeric, tokenizer, stop_remove, count_vec, idf, clean_up ])

In [16]:
cleaner = data_prep_pipe.fit(data)

In [17]:
clean_data = cleaner.transform(data)

In [18]:
clean_data = clean_data.select('label', 'features')

In [19]:
clean_data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(13424,[7,11,31,6...|
 0.0|(13424,[0,24,297,...|
 1.0|(13424,[2,13,19,3...|
 0.0|(13424,[0,70,80,1...|
 0.0|(13424,[36,134,31...|
 1.0|(13424,[10,60,139...|
 0.0|(13424,[10,53,103...|
 0.0|(13424,[125,184,4...|
 1.0|(13424,[1,47,118,...|
 1.0|(13424,[0,1,13,27...|
 0.0|(13424,[18,43,120...|
 1.0|(13424,[8,17,37,8...|
 1.0|(13424,[13,30,47,...|
 0.0|(13424,[39,96,217...|
 0.0|(13424,[552,1697,...|
 1.0|(13424,[30,109,11...|
 0.0|(13424,[82,214,47...|
 0.0|(13424,[0,2,49,13...|
 0.0|(13424,[0,74,105,...|
 1.0|(13424,[4,30,33,5...|
+-----+--------------------+
only showing top 20 rows

In [20]:
training, test = clean_data.randomSplit([0.7, 0.3])

In [21]:
spam_detector = nb.fit(training)

In [22]:
test_results = spam_detector.transform(test)

In [23]:
test_results.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(13424,[0,1,2,7,8...|[-785.03336620113...|[1.0,2.1854462308...| 0.0|
 0.0|(13424,[0,1,4,50,...|[-844.59735005487...|[1.0,4.7579551553...| 0.0|
 0.0|(13424,[0,1,9,14,...|[-537.34774230358...|[1.0,2.0483783875...| 0.0|
 0.0|(13424,[0,1,11,32...|[-890.64866397027...|[1.0,1.2559249020...| 0.0|
 0.0|(13424,[0,1,14,78...|[-687.21847831858...|[1.0,1.0301511616...| 0.0|
 0.0|(13424,[0,1,27,88...|[-1536.4586673265...|[1.80214376732331...| 1.0|
 0.0|(13424,[0,1,43,69...|[-614.61797872986...|[0.99995727290763...| 0.0|
 0.0|(13424,[0,1,874,1...|[-98.666872381222...|[0.99999995227856...| 0.0|
 0.0|(13424,[0,1,3657,...|[-143.84135386105...|[0.00574064303956...| 1.0|
 0.0|(13424,[0,2,3,6,9...|[-3286.9482109559...|[1.0,5.1058844193...| 0.0|
 0.0|(13424,[0,2,4,5,7...|[-986.46162571851...|[1.0,9.6775613374...| 0.0|
 0.0|(13424,[0,2,4,7,2...|[-519.75545645476...|[1.0,5.0335518608...| 0.0|
 0.0|(13424,[0,2,4,25,...|[-440.01701575024...|[1.0,2.6860551836...| 0.0|
 0.0|(13424,[0,2,4,44,...|[-1938.8240107737...|[1.0,5.4144993538...| 0.0|
 0.0|(13424,[0,2,4,128...|[-648.23210072182...|[1.0,4.9340145866...| 0.0|
 0.0|(13424,[0,2,7,8,1...|[-453.32226672656...|[1.0,5.7772069694...| 0.0|
 0.0|(13424,[0,2,7,11,...|[-731.29477277001...|[1.0,7.5630857162...| 0.0|
 0.0|(13424,[0,2,8,14,...|[-472.05550782245...|[1.0,2.0637595960...| 0.0|
 0.0|(13424,[0,2,8,25,...|[-310.72938945976...|[1.0,3.1409374057...| 0.0|
 0.0|(13424,[0,2,8,35,...|[-799.64326589572...|[0.99999999999876...| 0.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
acc_eval = MulticlassClassificationEvaluator()

In [26]:
acc = acc_eval.evaluate(test_results)

In [27]:
print(acc)

0.927057766992781